In [5]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [19]:
%run -i ../../zkstats/core.py

In [20]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

Here is a weird example where different col1 and col2 of data.json have different row number. We just want to show that different dimension is possible. In fact, a person can just request median(col_1), and median(col_2). Then just compute mean on his own as well, but here we show that the code is composable enough to do all at once. 

In [21]:

data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())
data1 = data['col_1']
data2 = data['col_2']


create_dummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())
dummy_data1 = dummy_data['col_1']
dummy_data2 = dummy_data['col_2']


dummy_theory_output_median1 = torch.tensor(np.median(dummy_data1))
dummy_lower_to_median1 = torch.tensor(np.sort(dummy_data1)[int(len(dummy_data1)/2)-1])
dummy_upper_to_median1 = torch.tensor(np.sort(dummy_data1)[int(len(dummy_data1)/2)])

dummy_theory_output_median2 = torch.tensor(np.median(dummy_data2))
dummy_lower_to_median2 = torch.tensor(np.sort(dummy_data2)[int(len(dummy_data2)/2)-1])
dummy_upper_to_median2 = torch.tensor(np.sort(dummy_data2)[int(len(dummy_data2)/2)])

dummy_theory_output_mean = torch.mean(torch.tensor([dummy_theory_output_median1, dummy_theory_output_median2]))

theory_output_median1 = torch.tensor(np.median(data1))
lower_to_median1 = torch.tensor(np.sort(data1)[int(len(data1)/2)-1])
upper_to_median1 = torch.tensor(np.sort(data1)[int(len(data1)/2)])

theory_output_median2 = torch.tensor(np.median(data2))
lower_to_median2 = torch.tensor(np.sort(data2)[int(len(data2)/2)-1])
upper_to_median2 = torch.tensor(np.sort(data2)[int(len(data2)/2)])

theory_output_mean = torch.mean(torch.tensor([theory_output_median1, theory_output_median2]))





In [22]:
def median(X, median, lower, upper):
    # since within 1%, we regard as same value
    count_less = torch.sum((X < 0.99*median).double())
    count_equal = torch.sum((torch.abs(X-median)<=torch.abs(0.01*median)).double())
    len = X.size()[1]
    half_len = torch.floor(torch.div(len, 2))
        
    # not support modulo yet
    less_cons = count_less<half_len+2*(len/2 - torch.floor(len/2))
    more_cons = count_less+count_equal>half_len

    # For count_equal == 0
    lower_exist = torch.sum((torch.abs(X-lower)<=torch.abs(0.01*lower)).double())>0
    lower_cons = torch.sum((X>1.01*lower).double())==half_len
    upper_exist = torch.sum((torch.abs(X-upper)<=torch.abs(0.01*upper)).double())>0
    upper_cons = torch.sum((X<0.99*upper).double())==half_len
    bound = count_less==half_len
    # 0.02 since 2*0.01
    bound_avg = (torch.abs(lower+upper-2*median)<=torch.abs(0.02*median))

    median_in_cons = torch.logical_and(less_cons, more_cons)
    median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        
    return(torch.where(count_equal==0, median_out_cons, median_in_cons), median)


In [23]:
def mean(X, mean):
    return (torch.abs(torch.sum(X)-X.size()[1]*(mean))<=torch.abs(0.01*X.size()[1]*mean), mean)

In [24]:
print("dummy output: ", dummy_theory_output_mean)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.median1 = nn.Parameter(data = dummy_theory_output_median1, requires_grad = False)
        self.lower1 = nn.Parameter(data = dummy_lower_to_median1, requires_grad = False)
        self.upper1 = nn.Parameter(data = dummy_upper_to_median1, requires_grad = False)
        self.median2 = nn.Parameter(data = dummy_theory_output_median2, requires_grad = False)
        self.lower2 = nn.Parameter(data = dummy_lower_to_median2, requires_grad = False)
        self.upper2 = nn.Parameter(data = dummy_upper_to_median2, requires_grad = False)
        self.mean = nn.Parameter(data = dummy_theory_output_mean, requires_grad = False)
    def forward(self,X1, X2):
        bool1, median1 = median(X1, self.median1, self.lower1, self.upper1)
        bool2, median2 = median(X2, self.median2, self.lower2, self.upper2)
        bool3, output_mean = mean(torch.tensor([median1, median2]).reshape(1,-1,1), self.mean)
        return (torch.logical_and(torch.logical_and(bool1, bool2),bool3), output_mean )

    

verifier_define_calculation(dummy_data_path, ['col_1', 'col_2'],sel_dummy_data_path,verifier_model, verifier_model_path)

dummy output:  tensor(5.3750, dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_4875/3449897344.py:17: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  bool3, output_mean = mean(torch.tensor([median1, median2]).reshape(1,-1,1), self.mean)
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_4875/3449897344.py:17: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  bool3, output_mean = mean(torch.tensor([median1, median2]).reshape(1,-1,1), self.mean)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/s

In [25]:
# prover calculates settings, send to verifier
print("theory mean output: ", theory_output_mean)

class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.median1 = nn.Parameter(data = theory_output_median1, requires_grad = False)
        self.lower1 = nn.Parameter(data = lower_to_median1, requires_grad = False)
        self.upper1 = nn.Parameter(data = upper_to_median1, requires_grad = False)
        self.median2 = nn.Parameter(data = theory_output_median2, requires_grad = False)
        self.lower2 = nn.Parameter(data = lower_to_median2, requires_grad = False)
        self.upper2 = nn.Parameter(data = upper_to_median2, requires_grad = False)
        self.mean = nn.Parameter(data = theory_output_mean, requires_grad = False)
    def forward(self,X1, X2):
        bool1, median1 = median(X1, self.median1, self.lower1, self.upper1)
        bool2, median2 = median(X2, self.median2, self.lower2, self.upper2)
        bool3, output_mean = mean(torch.tensor([median1, median2]).reshape(1,-1,1), self.mean)
        return (torch.logical_and(torch.logical_and(bool1, bool2),bool3), output_mean )

 
      
prover_gen_settings(data_path,['col_1', 'col_2'], sel_data_path, prover_model,prover_model_path, 'default', "resources", settings_path)

theory mean output:  tensor(49.3500, dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_4875/3663195370.py:18: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  bool3, output_mean = mean(torch.tensor([median1, median2]).reshape(1,-1,1), self.mean)
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_4875/3663195370.py:18: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  bool3, output_mean = mean(torch.tensor([median1, median2]).reshape(1,-1,1), self.mean)


==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-25518,25754],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":20992,"total_assignments":16092,"total_const_size":2432,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8,8],"module_sizes":{"kzg":[],"poseidon":[20992,[2]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [26]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output_mean)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 7.0805840492248535 seconds
Theory output:  tensor(49.3500, dtype=torch.float64)
!@# compiled_model exists? False
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.3515625
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [15295097400487804665, 12861486368330479023, 3350118022201779210, 343142782800691716], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10870267098303494893, 1752989342377741058, 8860763459400202009, 2635465469930673149]]], 'proof': '0e3222fa11e74d161cd51f4540cd7a494135a6a788cb50d4521ebf8db1b6902c1b78cd8e835c5894509ec43782e7986b298e39da3472abc05b6e8ec99a72912820de18faac76af5ea1fbdf9ef42208e23ae3df609703852819f84bf61a0f4ebe068f0beca18a03d279df65e523f93cad9c4daf91364d960b89e090beb21e8678191848ea81b0c2f526679d896d1e7bade846ce29146641b20f3645e06cf80f6504a13efcec9c4628d5cba04795076aa66723d2c0888181a986fe2206f3104cdf12836e21f04b2d3397576ba73fd681137f3960b502f4f37ebda42795eb0bb5bd02336d46124deb41756ccb4704a96cf421cb39285812d588ff7e9c766a1343642789f2610f2700c52a8768742c47e5a45df7fd0972f908aa5b4f11892ecb77050c946690db2a2164043335c31275c83c616c8aa69c

In [27]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  2
prf instances:  [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [15295097400487804665, 12861486368330479023, 3350118022201779210, 343142782800691716], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10870267098303494893, 1752989342377741058, 8860763459400202009, 2635465469930673149]]]
proof boolean:  1.0
proof result 1 : 49.3515625
verified
